# Prerequisite:
## Set Up Google Cloud account (billing, service, credentials)
- If you are new to Google Cloud, create an account. 
- In the Google Cloud console, create a Google Cloud project. 
- Select IAM & Admin service from within this project in the Google Cloud dashboard, create service account and generate service account key. 

## [Set up a Python development environment that run on Google Cloud](https://cloud.google.com/python/docs/setup)
- Install Python;
- Use `venv` to create isolated Python environment;  
- Install the Cloud Client Libraries for the Python development environment;
- [Install and initialize the gcloud CLI.](https://cloud.google.com/sdk/docs/install-sdk)

## [Install Vertex AI SDK for Python](https://cloud.google.com/vertex-ai/docs/python-sdk/use-vertex-ai-python-sdk) 
- Vertex AI is a comprehensive, managed machine learning platform by Google Cloud that allows ML engineers to develop and maintain ML/AL models.
- ```
  pip install --upgrade google-cloud-aiplatform
  ```
- Enable all recommended Vertex AI APIs from Vertex AI dashboard, e.g. Gemini Pro via the Gemini API.     

  
## [Quickstart with Gemini API with Python](https://ai.google.dev/tutorials/python_quickstart)
- You can either run Jupyter notebook on your local editor or run codes in Google Colab. 
- Getting-started/intro_gemini_python.ipynb [github](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_python.ipynb) | [More Ex1](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts?_gl=1*17is47x*_ga*MTY5MzM3MDc0My4xNjc3NjE4OTE4*_ga_WH2QY8WWF5*MTcxMTc1Mjg3Ni45LjEuMTcxMTc1Mzk1OS4wLjAuMA..&_ga=2.179362852.-1693370743.1677618918) | [Ex2](https://cloud.google.com/vertex-ai/generative-ai/docs/start/quickstarts/quickstart-multimodal)

# Import Libraries

In [22]:
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Image,
    Part,
)
from vertexai.vision_models import (
    Video
)

# Load the Gemini 1.0 Pro Vision Model
`geminii-1.0-pro-vision` is a multimodal model that supports multimodal prompts. You can include text, image(s) and video in your prompt requests and get text or code response. 


In [2]:
multimodal_model = GenerativeModel("gemini-1.0-pro-vision")

# Define helper functions
Define helper functions to load and display images.

In [59]:
import IPython
import time
import http.client
import typing
import urllib.request
import pathlib
import textwrap
import IPython.display
from IPython.display import display
from IPython.display import Markdown

from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps


def display_images(
    images: typing.Iterable[Image],
    max_width: int = 600,
    max_height: int = 350,
) -> None:
    for image in images:
        pil_image = typing.cast(PIL_Image.Image, image._pil_image)
        if pil_image.mode != "RGB":
            # RGB is supported by all Jupyter environments (e.g. RGBA is not yet)
            pil_image = pil_image.convert("RGB")
        image_width, image_height = pil_image.size
        if max_width < image_width or max_height < image_height:
            # Resize to display a smaller notebook image
            pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
        IPython.display.display(pil_image)


def get_image_bytes_from_url(image_url: str) -> bytes:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        image_bytes = response.read()
    return image_bytes


def load_image_from_url(image_url: str) -> Image:
    image_bytes = get_image_bytes_from_url(image_url)
    return Image.from_bytes(image_bytes)


def get_url_from_gcs(gcs_uri: str) -> str:
    # converts gcs uri to url for image display.
    url = "https://storage.googleapis.com/" + gcs_uri.replace("gs://", "").replace(
        " ", "%20"
    )
    return url


def print_multimodal_prompt(contents: list):
    """
    Given contents that would be sent to Gemini,
    output the full multimodal prompt for ease of readability.
    """
    for content in contents:
        if isinstance(content, Image):
            display_images([content])
        elif isinstance(content, Part):
            url = get_url_from_gcs(content.file_data.file_uri)
            IPython.display.display(load_image_from_url(url))
        else:
            print(content)


# Helper functions for handling videos contents 

* display video inline
* load video from uri
* generate response based on prompt  
* 

In [62]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


# Display video in-line

In [48]:
video_uri = "gs://vlm-videos/dump_trash.mp4"
video_url = "https://storage.cloud.google.com/" + video_uri.replace("gs://", "") + "?authuser=1"
IPython.display.Video(video_url)

# Load video from uri

In [ ]:
video = Part.from_uri(video_uri, mime_type="video/mp4")


# Generate text from a video file 

* Video must be passed in as a Cloud Storage URI starting with 'gs://' for `Part.from_uri(uri="gs://....)`.
* The bucket that stores the file must be in the same Google Cloud project that is sending the request. You must also specify the `mime_type` field. The supported MIME type for video includes `video/mp4`

In [ ]:
prompt = """
Answer the following questions using the video only:
What action was performed? 
"""
content = [prompt, video]
response = multimodal_model.generate_content(content)
to_markdown(response.text)

In [63]:
prompt = """
Answer the following questions using the video only:
What action was performed? 
"""
content = [prompt, video]
response = multimodal_model.generate_content(content)
to_markdown(response.text)

>  A hand came into the scene and threw the tomatoes into the trash one by one. Then the hand threw the empty bottle into the trash. Finally, the hand balled up the paper towel and threw it into the trash.

# Break down response 

In [83]:
actions = [a.strip() for a in response.text.split('.') if len(a) > 0]
actions

['A hand came into the scene and threw the tomatoes into the trash one by one',
 'Then the hand threw the empty bottle into the trash',
 'Finally, the hand balled up the paper towel and threw it into the trash']